## General script for cleaning text data

In [1]:
import re, string, timeit
import regex as re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm

In [2]:
# load data
df = pd.read_json("./Data/Memes/memesfinal_processed.json")
#df = df[df.date.between('2014-01-01', '2022-11-15')]
df['all_text'] = df['ocr_raw'] + ' ' + df['title_raw'] #+ ' ' + df['caption_raw']
df.head()

,id,score,num_comments,over18,url,date,title_raw,ocr_raw,caption_raw,all_text,processed,stemmed
0,2021.02.15_288,211152,1642,False,https://i.redd.it/n4ildkpurph61.png,2021-02-15,Wait I didn't mean it like that,High schooler: I'd kill for\ncollege tuition\n...,a man with a hat and a beard standing in front...,High schooler: I'd kill for\ncollege tuition\n...,"[wait, didnt, mean, like, high, schooler, kill...","[wait, didnt, mean, like, high, schooler, kill..."
1,2020.10.20_959,207206,811,False,https://i.redd.it/gsqgq6uwuau51.jpg,2020-10-20,Don't be mad,"@ renk. 5\n@ 1 Award\n\nI'm a dude, and | fuck...",a couple of white dogs standing next to each o...,"@ renk. 5\n@ 1 Award\n\nI'm a dude, and | fuck...","[dont, mad, renk, award, dude, fucking, hate, ...","[dont, mad, renk, award, dude, fuck, hate, peo..."
2,2020.01.18_675,207066,1734,False,https://i.redd.it/eg4t9kvlplb41.jpg,2020-01-18,For real tho,...,a cartoon of a cat with a box on its back,...,"[real, tho, cartoon, cat, box, back]","[real, tho, cartoon, cat, box, back]"
3,2020.06.09_755,187936,743,False,https://i.redd.it/1wniz8ionv351.jpg,2020-06-09,Right as rain after that,— :\nMy stomach hurts Bo i\n\n/@\nIt’s probabl...,a bunch of legos that are sitting on top of ea...,— :\nMy stomach hurts Bo i\n\n/@\nIt’s probabl...,"[right, rain, stomach, hurt, it’s, probably, g...","[right, rain, stomach, hurt, it’, probabl, gho..."
4,2020.07.08_336,182056,616,False,https://i.redd.it/nrj9smsfek951.jpg,2020-07-08,Licensed Dad Joke,Dads telling jokes at home\n\nDads cling es a ...,a collage of pictures of a dog wearing a birth...,Dads telling jokes at home\n\nDads cling es a ...,"[licensed, dad, joke, dad, telling, joke, home...","[licens, dad, joke, dad, tell, joke, home, dad..."


In [3]:
least_recent_date = df['date'].min(); print(least_recent_date)
most_recent_date = df['date'].max(); print(most_recent_date)

2018-01-01 00:00:00
2022-11-14 00:00:00


In [4]:
# new column for processing text
df['processed'] = df['all_text']

In [13]:
# NYT only: replace "G.O.P." type abreviations before removing punctuation
processed = []

for test_str in df['processed']:
    test_str = test_str + ' '
    while(test_str.find('.')>0):
        period = test_str.find('.')
        if test_str[period+1].isalpha():
            test_str = test_str[0:period] + test_str[period+1:len(test_str)]
        else:
            test_str = test_str[0:period] + ' ' + test_str[period+1:len(test_str)]
    processed.append(test_str)
df['processed'] = processed 

# NYT: replace NYT 
df['processed'] = df['processed'].str.replace("New York Times", "NYT")

### replace anything that is not a letter or number with a space

In [5]:
df['processed'] = df['processed'].str.replace("\n", " ") # this is for the ocr feature which returns \n chars
df['processed'] = df['processed'].str.replace("'s", " ")

In [6]:
df['processed'] = df['processed'].str.replace("[^a-zA-Z]+", " ") #nyt_df[column].map(lambda x: re.sub('[,\.!?]', '', x))

In [7]:
df['processed'] = df['processed'].str.translate(str.maketrans('', '', string.punctuation))

### make everything lowercase

In [8]:
df['processed'] = df['processed'].map(lambda x: x.lower()) 

### remove words less than 3 letters long

In [9]:
df['processed'] = df['processed'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

### tokenize

In [10]:
df['processed'] = df['processed'].apply(lambda x: x.split())

### remove stop words

In [10]:
# add additional stopwords to remove if desired
remove = ['ahead','yet','two','first','like','say','make','would','get','could','may','even','use','call','month',
         'week','day','want','need','long','come','three','also','know','made','see','monday','tuesday','wednesday',
         'thursday','friday','saturday','sunday','still','more','find','good','found','set','sinc','turn','place',
         'four','anoth','next','thing','second','well','five','ago','today','keep','go','put','among','think', 'new',
         'happen','seem','other','without','hour','word','tell','let','six','away','becam','leav','begin','time',
          'said','told']

In [11]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
#stop_words.extend(remove) # no additional stopwords were removed from memes data
tokenized_list_of_words = []

for i in tqdm(range(0, len(df))):
    l1 = df['processed'].iloc[i]
    tokenized_list_of_words.append([word for word in l1 if word not in stop_words])
    
df['processed'] = tokenized_list_of_words

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/katebarnes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|█████████████████████████████████| 899525/899525 [02:36<00:00, 5748.40it/s]


## replace specific words to improve the vocabulary agreement between the two datasets

In [12]:
# MEMES: if 'real' is not followed by 'estate' replace with 'actual'
updated_words = []
sublist = []
found = False

for l1 in tqdm(df['processed']):
    sublist = []
    for word in l1:
        if found==True:
            if word=='estate':
                sublist.append('real')
            else:
                sublist.append('actual')
            found = False
        if word=='real':
            found=True
            continue
        sublist.append(word)
    updated_words.append([sublist])
    
df['processed'] = updated_words

100%|████████████████████████████████| 899525/899525 [00:38<00:00, 23485.75it/s]


### lemmetize

In [13]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
lemma_list_of_words = []

for i in tqdm(range(0, len(df))):
    l1 = tokenized_list_of_words[i]
    lemma_list_of_words.append([lemmatizer.lemmatize(word) for word in l1])

df['processed'] = lemma_list_of_words

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/katebarnes/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
100%|█████████████████████████████████| 899525/899525 [01:58<00:00, 7615.24it/s]


### stem words to but store in a separate column just in case overstemming is a problem

In [14]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
stemma_list_of_words = []

for i in tqdm(range(0, len(df))):
    l1 = lemma_list_of_words[i]
    stemma_list_of_words.append([stemmer.stem(word) for word in l1])
    
df['stemmed'] = stemma_list_of_words

100%|█████████████████████████████████| 899525/899525 [04:44<00:00, 3164.23it/s]


### look into most common words

In [17]:
from collections import Counter

data = [' '.join(rec) for rec in df.stemmed]
data_set = ' '.join(data)

# split() returns list of all the words in the string
split_it = data_set.split()

# Pass the split_it list to instance of Counter class.
Counters_found = Counter(split_it)

# most_common() produces k frequently encountered
# input values and their respective counts.
most_occur = Counters_found.most_common(500)

In [22]:
most_occur[80:100]

[('take', 21879),
 ('messag', 21694),
 ('come', 21613),
 ('wear', 21155),
 ('got', 21108),
 ('friend', 21098),
 ('kid', 20932),
 ('boy', 20845),
 ('first', 20720),
 ('bunch', 20623),
 ('men', 20608),
 ('glass', 20207),
 ('beard', 20068),
 ('gun', 20024),
 ('back', 20003),
 ('anoth', 19246),
 ('photo', 19109),
 ('old', 19100),
 ('coupl', 19037),
 ('everi', 18914)]

### re-combine tokens into string

In [16]:
for i in range(len(df)):
    stemma_list_of_words[i] = ' '.join(stemma_list_of_words[i])
    
df['stemmed']= stemma_list_of_words

In [17]:
df.head()

,id,score,num_comments,over18,url,date,title_raw,ocr_raw,caption_raw,all_text,processed,stemmed
0,2021.02.15_288,211152,1642,False,https://i.redd.it/n4ildkpurph61.png,2021-02-15,Wait I didn't mean it like that,High schooler: I'd kill for\ncollege tuition\n...,a man with a hat and a beard standing in front...,High schooler: I'd kill for\ncollege tuition\n...,"[high, schooler, kill, college, tuition, milit...",high schooler kill colleg tuition militari rec...
1,2020.10.20_959,207206,811,False,https://i.redd.it/gsqgq6uwuau51.jpg,2020-10-20,Don't be mad,"@ renk. 5\n@ 1 Award\n\nI'm a dude, and | fuck...",a couple of white dogs standing next to each o...,"@ renk. 5\n@ 1 Award\n\nI'm a dude, and | fuck...","[renk, award, dude, fucking, hate, people, tel...",renk award dude fuck hate peopl tell look tire...
2,2020.01.18_675,207066,1734,False,https://i.redd.it/eg4t9kvlplb41.jpg,2020-01-18,For real tho,...,a cartoon of a cat with a box on its back,...,"[real, tho]",real tho
3,2020.06.09_755,187936,743,False,https://i.redd.it/1wniz8ionv351.jpg,2020-06-09,Right as rain after that,— :\nMy stomach hurts Bo i\n\n/@\nIt’s probabl...,a bunch of legos that are sitting on top of ea...,— :\nMy stomach hurts Bo i\n\n/@\nIt’s probabl...,"[stomach, hurt, it’s, probably, ghost, ri, any...",stomach hurt it’ probabl ghost ri anyway here’...
4,2020.07.08_336,182056,616,False,https://i.redd.it/nrj9smsfek951.jpg,2020-07-08,Licensed Dad Joke,Dads telling jokes at home\n\nDads cling es a ...,a collage of pictures of a dog wearing a birth...,Dads telling jokes at home\n\nDads cling es a ...,"[dad, telling, joke, home, dad, cling, work, “...",dad tell joke home dad cling work “et licens d...


### save data

In [97]:
save = df[['id','stemmed']]
save.to_json('./Data/Memes/memesfinal_processed.json')

In [98]:
check = pd.read_json('./Data/Memes/memestitle_oc.json')
check.head()

,id,score,num_comments,over18,url,date,title_raw,ocr_raw,caption_raw,all_text,processed,stemmed
0,2021.02.15_288,211152,1642,False,https://i.redd.it/n4ildkpurph61.png,2021-02-15,Wait I didn't mean it like that,High schooler: I'd kill for\ncollege tuition\n...,a man with a hat and a beard standing in front...,Wait I didn't mean it like that - High schoole...,"[wait, didnt, mean, like, high, schooler, kill...","[wait, didnt, mean, like, high, schooler, kill..."
1,2020.10.20_959,207206,811,False,https://i.redd.it/gsqgq6uwuau51.jpg,2020-10-20,Don't be mad,"@ renk. 5\n@ 1 Award\n\nI'm a dude, and | fuck...",a couple of white dogs standing next to each o...,Don't be mad - @ renk. 5\n@ 1 Award\n\nI'm a d...,"[dont, mad, renk, award, dude, fucking, hate, ...","[dont, mad, renk, award, dude, fuck, hate, peo..."
2,2020.01.18_675,207066,1734,False,https://i.redd.it/eg4t9kvlplb41.jpg,2020-01-18,For real tho,...,a cartoon of a cat with a box on its back,For real tho - ...,"[real, tho, cartoon, cat, box, back]","[real, tho, cartoon, cat, box, back]"
3,2020.06.09_755,187936,743,False,https://i.redd.it/1wniz8ionv351.jpg,2020-06-09,Right as rain after that,— :\nMy stomach hurts Bo i\n\n/@\nIt’s probabl...,a bunch of legos that are sitting on top of ea...,Right as rain after that - — :\nMy stomach hur...,"[right, rain, stomach, hurt, it’s, probably, g...","[right, rain, stomach, hurt, it’, probabl, gho..."
4,2020.07.08_336,182056,616,False,https://i.redd.it/nrj9smsfek951.jpg,2020-07-08,Licensed Dad Joke,Dads telling jokes at home\n\nDads cling es a ...,a collage of pictures of a dog wearing a birth...,Licensed Dad Joke - Dads telling jokes at home...,"[licensed, dad, joke, dad, telling, joke, home...","[licens, dad, joke, dad, tell, joke, home, dad..."
